In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle('data/micro_sud3_normalized.pkl')
df = df.reset_index()
df.head()

def split_dataframe(dataframe, percent):
    nb_rows = int(np.floor(percent * len(dataframe)))
    return dataframe[:nb_rows], dataframe[nb_rows:]
    """
def dataframe_to_xy(df, sequence_size):

    out_X = np.zeros((len(df)//sequence_size, sequence_size, 8))
    out_y = np.zeros((len(df)//sequence_size, sequence_size))
    i = 0
    while i + sequence_size < len(df):
        sequence = df.iloc[i:i+sequence_size]
        out_X[i//sequence_size] =  np.array(sequence[['NO2_61FD', 'NO2_61F0', 'NO2_61EF', 'temp', 'rh',\
                                 'tgrad', 'pressure', 'pluvio']])
        out_y[i//sequence_size] = np.array(sequence['NO2_ref'])
        i += sequence_size
        
    return out_X, out_y
    """

def dataframe_to_xy(df):
    return (np.array(df[['NO2_61FD', 'NO2_61F0', 'NO2_61EF', 'temp', 'rh',\
                         'tgrad', 'pressure', 'pluvio']]),\
            np.array(df['NO2_ref']))

        
df_train, df_test = split_dataframe(df, 0.5) 
df_valid, df_test = split_dataframe(df_test, 0.5)

X_train, y_train = dataframe_to_xy(df_train)
X_valid, y_valid = dataframe_to_xy(df_valid)
X_test, y_test = dataframe_to_xy(df_test)

X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_valid= X_valid.reshape((X_valid.shape[0], 1,  X_valid.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

#y_train = y_train.reshape((1, len(y_train)))
#y_valid = y_valid.reshape((1, len(y_valid)))
#y_test = y_test.reshape((1, len(y_test)))

def dataframe_to_xy_sequences(df, sequence_size):
    out_X = np.zeros((len(df)//sequence_size, sequence_size, 8))
    out_y = np.zeros((len(df)//sequence_size, sequence_size))
    i = 0
    while i + sequence_size < len(df):
        sequence = df.iloc[i:i+sequence_size]
        out_X[i//sequence_size] =  np.array(sequence[['NO2_61FD', 'NO2_61F0', 'NO2_61EF', 'temp', 'rh',\
                                 'tgrad', 'pressure', 'pluvio']])
        out_y[i//sequence_size] = np.array(sequence['NO2_ref'])
        i += sequence_size
        
    return out_X, out_y

In [3]:
from keras.layers import SimpleRNN, Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping

def simple_rnn_model(nb_units, input_dim, loss='mean_squared_error', optimizer='adam'):
    print(input_dim)
    model = Sequential()
    model.add(SimpleRNN(nb_units, input_shape=input_dim))#input_dim=input_dim[1], input_length=input_dim[0], return_sequences=True))
    #model.add(Dense(nb_units))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss=loss, optimizer=optimizer)
    model.summary()
    return model

Using TensorFlow backend.


In [ ]:
model = simple_rnn_model(16, X_train.shape[1:])

(1, 8)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 16)                400       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, mode='auto', patience=10)
history = model.fit(X_train, y_train, batch_size=32, epochs=10000, validation_data=(X_valid, y_valid), callbacks=[early_stopping], verbose=1, shuffle=False)

Train on 1126 samples, validate on 563 samples
Epoch 1/10000
1126/1126 [==============================] - 0s - loss: 3352.5525 - val_loss: 3015.8167
Epoch 2/10000
1126/1126 [==============================] - 0s - loss: 3342.8907 - val_loss: 3007.8665
Epoch 3/10000
1126/1126 [==============================] - 0s - loss: 3331.3117 - val_loss: 2998.1805
Epoch 4/10000
1126/1126 [==============================] - 0s - loss: 3316.6519 - val_loss: 2986.1466
Epoch 5/10000
1126/1126 [==============================] - 0s - loss: 3298.6062 - val_loss: 2971.3558
Epoch 6/10000
1126/1126 [==============================] - 0s - loss: 3277.0715 - val_loss: 2953.5259
Epoch 7/10000
1126/1126 [==============================] - 0s - loss: 3251.8953 - val_loss: 2932.4591
Epoch 8/10000
1126/1126 [==============================] - 0s - loss: 3222.9662 - val_loss: 2907.9869
Epoch 9/10000
1126/1126 [==============================] - 0s - loss: 3190.3740 - val_loss: 2880.1756
Epoch 10/10000
1126/1126 [=========

1126/1126 [==============================] - 0s - loss: 1269.2637 - val_loss: 1102.9789
Epoch 81/10000
1126/1126 [==============================] - 0s - loss: 1256.2512 - val_loss: 1092.5091
Epoch 82/10000
1126/1126 [==============================] - 0s - loss: 1243.4040 - val_loss: 1082.2287
Epoch 83/10000
1126/1126 [==============================] - 0s - loss: 1230.7200 - val_loss: 1072.1304
Epoch 84/10000
1126/1126 [==============================] - 0s - loss: 1218.1967 - val_loss: 1062.2074
Epoch 85/10000
1126/1126 [==============================] - 0s - loss: 1205.8309 - val_loss: 1052.4530
Epoch 86/10000
1126/1126 [==============================] - 0s - loss: 1193.6185 - val_loss: 1042.8603
Epoch 87/10000
1126/1126 [==============================] - 0s - loss: 1181.5554 - val_loss: 1033.4218
Epoch 88/10000
1126/1126 [==============================] - 0s - loss: 1169.6370 - val_loss: 1024.1309
Epoch 89/10000
1126/1126 [==============================] - 0s - loss: 1157.8586 - val_l

1126/1126 [==============================] - 0s - loss: 533.0903 - val_loss: 484.6076
Epoch 161/10000
1126/1126 [==============================] - 0s - loss: 527.1262 - val_loss: 479.4060
Epoch 162/10000
1126/1126 [==============================] - 0s - loss: 521.2456 - val_loss: 474.3009
Epoch 163/10000
1126/1126 [==============================] - 0s - loss: 515.4482 - val_loss: 469.2914
Epoch 164/10000
1126/1126 [==============================] - 0s - loss: 509.7335 - val_loss: 464.3766
Epoch 165/10000
1126/1126 [==============================] - 0s - loss: 504.1009 - val_loss: 459.5552
Epoch 166/10000
1126/1126 [==============================] - 0s - loss: 498.5500 - val_loss: 454.8266
Epoch 167/10000
1126/1126 [==============================] - 0s - loss: 493.0800 - val_loss: 450.1896
Epoch 168/10000
1126/1126 [==============================] - 0s - loss: 487.6906 - val_loss: 445.6429
Epoch 169/10000
1126/1126 [==============================] - 0s - loss: 482.3811 - val_loss: 441.1

1126/1126 [==============================] - 0s - loss: 260.0683 - val_loss: 271.7236
Epoch 241/10000
1126/1126 [==============================] - 0s - loss: 258.3060 - val_loss: 270.3046
Epoch 242/10000
1126/1126 [==============================] - 0s - loss: 256.5614 - val_loss: 268.8939
Epoch 243/10000
1126/1126 [==============================] - 0s - loss: 254.8342 - val_loss: 267.4916
Epoch 244/10000
1126/1126 [==============================] - 0s - loss: 253.1240 - val_loss: 266.0981
Epoch 245/10000
1126/1126 [==============================] - 0s - loss: 251.4307 - val_loss: 264.7134
Epoch 246/10000
1126/1126 [==============================] - 0s - loss: 249.7537 - val_loss: 263.3381
Epoch 247/10000
1126/1126 [==============================] - 0s - loss: 248.0930 - val_loss: 261.9722
Epoch 248/10000
1126/1126 [==============================] - 0s - loss: 246.4483 - val_loss: 260.6163
Epoch 249/10000
1126/1126 [==============================] - 0s - loss: 244.8194 - val_loss: 259.2

In [ ]:
y_pred = model.predict(X_test)
plt.plot(history.history['loss'], 'r-')
plt.plot(history.history['val_loss'], 'b-')
plt.show()

plt.title('y_pred, y_test')

plt.plot(y_pred[:], y_test[:], '+')
plt.show()

In [ ]:
X_train, y_train = dataframe_to_xy_sequences(df_train, 4)
X_valid, y_valid = dataframe_to_xy_sequences(df_valid, 4)
X_test, y_test = dataframe_to_xy_sequences(df_test, 4)

def simple_rnn_model(nb_units, input_dim, loss='mean_squared_error', optimizer='adagrad'):
    print(input_dim)
    model = Sequential()
    model.add(SimpleRNN(nb_units, batch_input_shape=(96, 4, 8)))#input_dim=input_dim[1], input_length=input_dim[0], return_sequences=True))
    #model.add(Dense(nb_units))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss=loss, optimizer=optimizer)
    model.summary()
    return model

In [ ]:
model_2 = simple_rnn_model(16, X_train.shape[1:])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose=1, mode='auto', patience=10)
history = model.fit(X_train, y_train, batch_size=32, epochs=5000, validation_data=(X_valid, y_valid), callbacks=[early_stopping], verbose=1)

In [ ]:
X_train.shape